In [ ]:
# MSanalyst documentation
MSanalyst offers a [graphical user interface](https://msanalyst.net/) to annotate molecular features from LC/MS data.
## Installation
### Installing from PyPI or github
`pip install msanalyst`
or
`git clone git@github.com:WenchYu/MSanalyst.git`
Due to the size of the library files, they need to download [here](https://drive.google.com/file/d/1w6HF3w1KIJlTz_QaVqqtN1BzkGDhDgzw/view?usp=sharing) and unzip to the msanalyst directory.
```
MSanalyst/
│
├── msdb/
│   ├── Unzip the compressed library files here
│   └── ...
├── ...
└── ...
```

### Use the web version
If you have trouble installing MSanalyst and please use the [web version](https://msanalyst.net/).

In [ ]:
# Data analysis with MSanalyst
Msanalyst has four main functional modules, the `Molecular Networking` as main pipeline for mapping the overall profile of the metabolomic features, the `Reanalysis` to quickly find a suitable combination of parameters and algorithms, the `Spectrum Searching` section to perform single MS comparison against the entire MSanalyst library , and the `Customed Database` allowing users to generate their own MS library for special purpose.

## Molecualr Networking
This is the main pepline of MSanalyst. Annotations are obtained by MS comparison and structural infomation can be propagated through edges in molecular networks.

### Data requirements
Feature quantification table (CSV format) and MS<sup>2</sup> spectral file (MGF format) are needed. In brief, data conversion is the first step to generate standardized MS data(mzML or mzXML format). MSconvert can handle all the raw data from different vendors, except for Burker using its own DataAnalysis software to fullfill the conversion.
We recommend MZmine to preprocess the converted 'mzML' or 'mzxML' data, including five necessary steps: mass detection, chromatogram builder, chromatogram deconcolution, feature alignment and feautre export. Details in [MZmine documentation](https://mzmine.github.io/mzmine_documentation/workflows/lcmsworkflow/lcms-workflow.html) about untargeted LC-MS workflow.

### Usage
You can use the command:
```
python main.py  -q ./example/example_quant.csv -m ./example/example.mgf -o ./example/
```
or click the 'Molecular Networking' button from the MSanalyst main page:
<img src='./mn_button.jpg' alt='mn_button' width='200'>
This will bring you to the workflow to start you analysis.
<img src="./MN_module.jpg" alt="MN_module" width="500">


A result file will be generated at the specific directory by the `-o` option and named after by basename of the 'example_quant.csv'. The result folder structure are listed as follows:

<img src="./MN_result.jpg" alt="MN_result" width="500">

The description of every kind of file is listed as follows.

1. Folders named by 'row ID': These numbers are the same as the 'row ID' column within Feature quantification table input by `-q` option. In each folder, every MS2 comparison result and related spectra (MGF format) are kept for further investigation. The detailed information (e.g. reference id, spectral similarity) are recorded in the corresponding table with E representing experimental library and IS representing in-silico library.
<img src="./node_results.jpg" alt="node_results" width="500">

2. 'E_MS1match_example_quant.csv', 'IS_MS1match_example_quant.csv' are the summary of match results in each folder.

3. `example_modified_cosine_0.7_5.graphml` is the network file and can be directly drag into Cytoscape for visualization. We provide a template, using preset colors and shapes to display node and edge information.
This network was generated using four spectral similarity algorithms, neutral loss and symmetric chi squared for clustering and modified cosine and peak percentage for annotating.
features are categorized into different levels based on the library search results. Specifically, if no MS1 match is found within the defined mass tolerance (e.g. 5 ppm), features will be annotated as category A, indicating the highest level of novelty. MS1 matched features will be further compared at MS2 level and classified into level B (matched by experimental spectra) or level C (matched by in-silico spectra). Depending on the user-defined MS2 similarity threshold and matched peaks count (MPC), it can be further categorized as level 1 (meeting the defined thresholds) or 2 (not meeting the thresholds). For instance, category C1 features only have in-silico MS1 match but no MS2 match, implying only the theoretical MS1 of intrest could be found with limited structural information.

<img src="./MN_MCS.jpg" alt="MN_MCS" width="500">

### Parameters
MSanalyst provides all the parameters for each step involving in the analysis process. These parameters are shared by all the modules.
```
-q, --quant_file, help = feature quantification table (csv format) exported by MZmine
-m, --mgf_file, help = tandem mass spectral file (mgf format) exported by MZmine
-o, --output, help = output path
-i1f, --isms1_file, help = in-silico ms1 file
-e1f, --edbms1_file, help = experimental ms1 file
-i2f, --isms2_file, help = in-silico  library
-e2f, --edbms2_file, help = experimental ms2 library

Library matching parameters
-pmt, --pepmass_match_tolerance, help = allowed ppm to
lerance in MS1 matching, default = 5
-lmm, --library_matching_method, help = sepctral similarity algorithm using in library matchbing, default = modified_cosine
-islms, --is_library_matching_similarity, help = threshold of in-silico library matching similarity, default = 0.7
-islmp, --is_library_matching_peaks, help= threshold of in-silico library matching shared peaks, default = 5
-lms, --library_matching_similarity, help = threshold of experimental library matching similarity, default = 0.7
-lmp, --library_matching_peaks, help = threshold of experimental library matching shared peaks, default = 5
-ppt, --peak_percentage_threshold, help = Library matching shared peaks percentage threshold, default = 0.7

Clustering parameters
-scm, --self_clustering_method, help = method for self-clustering, default = modified_cosine
-scs, --self_clustering_similarity, help= threshold of self clustering similarity, default = 0.7
-scp, --self_clustering_peaks, help= threshold of self clustering shared peaks, default = 5
-topk, --top_k, help = maximum degree of a node (maximum egdes a node allowed to have), default=10

Others
-qms1, --query_ms1, help = precusor mass search against entire MSanalyst library
-sc, --spectrum_clean, help = if need to clean spectra (e.g. noise removing, standardizaion, default=True
-li,--library_info, help = information (csv/tsv/xlsx format) of standards used for creating the customed library
-mn1, --mn1_file, type=str, help = molecular network file 1
-mn2, --mn2_file, type=str, help = molecular network file 2
```

## Reanalysis
Since the tandem mass largely differ from diverse mass spectrometry, collision energy and compound classes. With this module, you can quickly adjust the settings during mass comparison, such as mass tolerance, spectral similarity thresholds, spectral similarity algorithms, to find the most suitable combination for your data.
### Data requirements
Completed `molecular networking` results and the same in-/output parameters (`-q`, `-m` and `-o`) are required.
### Usage
Here's an example of command:
```
python re-networking.py -m ./example/example.mgf -q ./example/example_quant.csv -scm neutral_loss -scs 0.1 -scp 1
```
or click 'Reanalysis' button on the website:
<img src='./reanalysis_button.jpg' alt='reanalysis_button' width='200'>

to go to the page:
<img src="./reanalysis_page.jpg" alt="reanalysis_moudle" width="500">

A new '.graphml' file will be generated.

<img src='./81740318135_.pic.jpg' alt='new_graphml' width='500'>

Additional edges formed by neutral_loss algorithms are found.

<img src='./MN_NL.jpg' alt='MN_NL' width='500'>

## Spectrum Searching
This function could be useful, if you want to check a single feature.
<img src='./SpeSearch_button.jpg' alt='SpeSearch_button' width='200'>
### Usage
You can do spectrum search in command:
```
python ms1search.py -qms1 319.2339
python ms2search.py -m 319.mgf

```
Type the precursor mass or drop the file:
<img src='./SpeInput.jpg' alt='SpeInput' width='500'>
The result is similar with the result of `Molucular Networking`,except for network file and ID folders.
<img src= './SpeSearch_result.jpg' alt='SpeSearch_result' width='500'>

## Customed Database
Msanalyst allows users to annotate data by creating their own MS libraries.

<img src='./CusDB_button.jpg' alt='CustomDB_button' width='200'>

### Data requirements
The MZmine-generated quantification table (csv format) and spectral file (mgf format) can be used to generate the customed MS library with only minor changes. A table (tsv/csv/xlsx file are supported) is needed to record the specific information of the standards or target compounds (e.g. smiles, adduct, organisms). Users can directly copy the target spectra to generate new 'mgf' file. Please make sure that the 'feature_id' recorded in the table corresponds to them in 'mgf' file one by one.
<img src='./CusDB_requirement.jpg' alt='CusDB_requirement' width='200'>

### Usage
You can use the command:
```
python customized_db.py -m ./customed_db/DB.mgf -li ./customed_db/DB.xlsx -o ./customed_db
python main.py  -e1f ./customed_db/DB_ms1.csv -e2f ./customed_db/DB_ms2.json -q ./example/example_quant.csv -m ./example/example.mgf -o ./example/
```
Two library files will be generated by `customized_db` and directly used in `main` pipeline.
When using online tools,the database can be selected in other modules after library generation.
<img src='./CusDB_moudle.jpg' alt='CusDB_moudle' width='500'>

## Network merging
Currently, only command line tool support this function.
```
python mn_merging.py -mn1 ./example/example_quant_result/example_modified_cosine_0.7_5.graphml -mn2 ./example/example_quant_result/example_modified_cosine_0.1_1.graphml -o ./example/
```

<img src='./MergeResFile.jpg' alt='MergeResFile' width='500'>

Different spectral similarity algorithms can  provide additional
information into the relationships and annotations.

<img src='./MN_merging.jpg' alt='MN_merging' width='500'>



# Mass spectral similarity algorithms
The spectral similarity algorithms are collected from [Spectral Entropy](https://spectralentropy.readthedocs.io/en/master/#spectral_similarity.multiple_distance) and [Spectrum_utils](https://spectrum-utils.readthedocs.io/en/latest/quickstart.html)

Mass spectra are preprocessed by default, including following procedures:

1. Remove ions have *m/z* higher than the precusor mass.

2. Centroid peaks by merging peaks within a given mass tolerance.

3. Remove ions have intensity lower than max intensity * fixed value.

Distance between two MS$^2$ spectra are calculated as follows:

1. Find common peaks (*m/z*) within a given mass tolerance.

2. Define intensity from spec1 as P and intensity from spec2 as Q.

Similarity = 1 - Distance

*The spectral similarity algorithms have been demonstrated to exhibit varying degrees of effectiveness comparing with classic cosine-based similarity. However Collision energy and the differences in mass spectrometries can vary widely in practical applications, especially in untargeted analyses where the sample composition is unknown. Therefore, we recommend using all algorithms for clustering to identify effective algorithm combinations that best suit the specific dataset.
*

## dot_product

The similarity score is defined as:

$Distance = \frac{\sum Q_i P_i}{\sqrt{\sum Q_i^2} \sqrt{\sum P_i^2}}$

## absolute_value_distance
$Distance = \frac { \sum(|Q_i-P_i|)}{\sum P_i})$

For each pair of intensities $Q_i$ and $P_i$, it reflects the intensity difference between the two spectra at each data point. Sum up all absolute differences$|Q_i-P_i|$ to obtain the total absolute difference



### Application Note
#### Sensitivity to Outliers:
This method is sensitive to outliers since a single large difference can significantly affect the total difference.

##### Lack of Correlation Consideration:
It does not account for correlations between fragment ions, focusing solely on intensity differences.

## avg_l_distance
$Distance = \frac{1}{2}(\sum|P_i-Q_i|+\underset{i}{\max}{|P_i-Q_i|})$

$(\sum|P_i-Q_i|)$ evaluates the cumulative intensity difference across all data points, providing a measure of overall similarity between the two spectra.

By including the largest single-point intensity difference ($(\underset{i}{\max}{|P_i-Q_i|})$), this formula emphasizes significant outliers or peaks that might represent key differences between the spectra.

Combining these two metrics ensures the score reflects both overall similarity and the presence of critical intensity mismatches, which can be important for identifying unique spectral features.

### Application notes
#### Robustness to Outliers:
Unlike simpler metrics such as the mean absolute difference, this formula explicitly accounts for the maximum deviation, making it more sensitive to outliers. This can be useful when critical spectral features are concentrated in a few peaks.

#### Highlighting Key Peaks:
By emphasizing the largest differences, the method is particularly effective when the analysis aims to identify unique or significant features in MS2 spectra, such as marker compounds or structural motifs.

#### Normalization:
As with other similarity measures, normalization or standardization may be required to prevent bias from differences in overall intensity scales between spectra.

### Limitations:
The inclusion of the maximum difference may exaggerate the impact of single outlier peaks, potentially skewing similarity assessments if these peaks are not meaningful. Proper preprocessing, such as noise reduction and peak alignment, is essential.

## baroni_urbani_buser_distance

$Distance = 1-\frac{\sum\min{(P_i,Q_i)}+\sqrt{\sum\min{(P_i,Q_i)}\sum(\max{(P)}-\max{(P_i,Q_i)})}}{\sum{\max{(P_i,Q_i)}+\sqrt{\sum{\min{(P_i,Q_i)}\sum(\max{(P)}-\max{(P_i,Q_i)})}}}}$

## bhattacharya_1_distance
$Distance = (\arccos{(\sum\sqrt{P_{i}Q_{i}})})^2$


The geometric mean (\sqrt{P_{i}Q_{i}}$) focuses on the proportional similarity between two spectra at each data point. It is sensitive to the alignment of relative intensities, reducing the impact of absolute differences.

Summing the geometric means aggregates the pointwise similarities, while squaring amplifies the importance of higher alignment across multiple peaks.

The arccosine function provides a bounded angular representation of the similarity, where smaller values indicate greater similarity.

### Application notes
#### Proportional Similarity Emphasis:
This method emphasizes the relative relationship between intensities rather than absolute values. It is particularly useful when comparing spectra with different total intensities or baseline noise levels.
#### Sensitivity to Alignment:
By leveraging the geometric mean, the formula strongly rewards spectra with closely aligned peak intensities, making it effective for identifying structurally related compounds.
#### Normalization:
The method inherently reduces the need for additional normalization because it focuses on proportional intensities rather than absolute differences.
#### Limitations:
Highly sensitive to misaligned peaks, as geometric means drop to near-zero for unaligned intensities.


## bhattacharya_2_distance

$Distance = -\ln{(\sum\sqrt{P_{i}Q_{i}})}$

The same geometric mean and summation as bhattacharya_1_distance but with natural logarithm $\ln$ treatment

### Application notes
#### Proportional Alignment Sensitivity:
This metric highlights relative intensity alignment between spectra. It is well-suited for analyzing spectra with differing absolute intensities, such as those resulting from variable ionization efficiencies.

#### Scale Compression:
The logarithmic transformation helps to manage the influence of large sums, making the similarity score robust to outliers or high-intensity peaks.








## canberra_distance

$Distance = \sum\frac{|P_{i}-Q_{i}|}{|P_{i}|+|Q_{i}|}$

evaluates the pairwise proportional difference between corresponding intensity values of two spectra. By dividing the absolute difference $|P_i-Q_i|$ by the sum of absolute intensities $|P_i|+|Q_i|$, it normalizes the differences, ensuring the metric is scale-independent. The sum aggregates these normalized differences across all data points.

### Application Notes
#### Proportional Difference Emphasis:
This metric is particularly effective when the goal is to compare the relative intensity distributions of two spectra, regardless of their absolute scale.
#### Scale Independence:
By normalizing differences against the combined intensity, the score is robust to variations in total intensity, making it suitable for comparing spectra obtained under varying experimental conditions.


## chebyshev_distance
$Distance = \underset{i}{\max}{(|P_{i}\ -\ Q_{i}|)}$

This approach calculates the maximum absolute intensity difference between two spectra at any peak. It identifies the single largest discrepancy between corresponding points in intensity $P$ and $Q$.

###  Application notes
#### Sensitive to single peaks:
This metric emphasizes the largest difference between two spectra and sensitive to single peaks.
#### focusing on specific features:
Unlike metrics that average differences across all points, this measure highlights the most extreme divergence, focusing on specific features of the spectra. May overlook small, cumulative differences across the spectra that are significant for overall similarity metrics.



## clark_distance
$Distance = (\frac{1}{N}\sum(\frac{P_i-Q_i}{|P_i|+|Q_i|})^2)^\frac{1}{2}$

## cosine_distance
$Distance = \sqrt{\frac{(\sum{Q_iP_i})^2}{\sum{Q_i^2\sum P_i^2}}}$

It is the same as dot_product_distance.

## dice_distance
$Distance = \frac{\sum(P_i-Q_i)^2}{\sum P_i^2+\sum Q_i^2}$
This metric computes the normalized sum of squared differences between corresponding intensities in two spectra. The denominator sums the squared intensities of both spectra, ensuring normalization.

### Application notes
#### Sensitivity to Differences:
The squared difference enhances sensitivity to significant discrepancies, making it suitable for detecting major deviations in spectral patterns.
#### Overweighting large deviations:
Squaring amplifies the influence of larger differences, which can overshadow minor but meaningful variations in spectra.








## divergence_distance
$Distance = 2\sum\frac{(P_i-Q_i)^2}{(P_i+Q_i)^2}$

### Application notes
#### Balanced effect:
Squaring the differences could highlight the discrepancies between the two spectra and the presence of ${(P_i+Q_i)^2}$ in the denominator might balance the influence of intensity differences.
#### Interpretability:
The formula does not directly account for the overall intensity or distribution of the spectra, which could affect the interpretation of the comparison results.


## dot_product_reverse_distance
$Distance = 1 - \sqrt{\frac{(\sum{{} {P_i^{'}}})^2}{{\sum{(Q_i^{'})^2}{\sum (P_i^{'})^2}}}}$
$P^{'}_{i}=\frac{P^{''}_{i}}{\sum_{i}{P^{''}_{i}}}$,
$P^{''}_{i}=\begin{cases}
        0 & \text{ if } Q_{i}=0 \\
        P_{i} & \text{ if } Q_{i}\neq0
        \end{cases}
$



## entropy_distance

$Distance = -\frac{2\times S_{PQ}^{'}-S_P^{'}-S_Q^{'}} {ln(4)}$
$S_I^{'}=\sum_{i} {I_i^{'} ln(I_i^{'})}, I^{'}=I^{w}$

with$\ w=0.25+S\times 0.5\ (S<1.5)$



## euclidean_distance

$Distance = (\sum|P_{i}-Q_{i}|^2)^{1/2}$

calculates the Euclidean distance (also known as L2 norm) between two mass spectra.

### Application notes
#### Sensitivity to Outliers:
The Euclidean distance is sensitive to outliers because squaring the differences amplifies the effect of large deviations. If outliers are a concern, consider methods to robustify the distance calculation against outliers.
#### Equal Weighting:
All dimensions (peaks in the spectra) are given equal weight in the calculation. In some cases, this might not be desirable if certain peaks are more informative than others.

## fidelity_distance
$Distance = 1-\sum\sqrt{P_{i}Q_{i}}$

This formula calculates a similarity measure by summing the square roots of the product of corresponding intensity values from two spectra, and then subtracting this sum from 1. The idea might be to capture some form of interaction or correlation between the two sets of data.

### Application notes
#### Interaction Capture:
By multiplying and summing the square roots of the intensities, the formula might capture some form of interaction effect between the spectral data points.

#### Sensitivity to Outliers:
Similar to other measures involving square roots or multiplication, this formula might be sensitive to outliers.

#### Scale Dependence:
The formula is dependent on the scale of the data.$P_i$and$Q_i$ need to be on the same scales.

## harmonic_mean_distance
$Distance = 1-2\sum(\frac{P_{i}Q_{i}}{P_{i}+Q_{i}})$

is a variation of the [Tanimoto coefficient](https://en.wikipedia.org/wiki/Jaccard_index), which is a measure of similarity between two sets or vectors.

### Application note
#### Balanced Measure:
The formula takes into account both the presence and absence of peaks by considering the product $P_iQ_i$ and the sum $P_i+Q_i$. Unlike some other similarity measures, this formula can handle zero values, as the division by $|P_i+Q_i|$ prevents division by zero.


## hellinger_distance
$Distance = \sqrt{2\sum(\sqrt{\frac{P_i}{\bar{P}}}-\sqrt{\frac{Q_i}{\bar{Q}}})^2}$

calculates the distance between two mass spectra by considering the normalized intensities of their peaks. ${\bar{P}}$ and ${\bar{Q}}$ represent the mean intensities.

## Application notes
### Sensitivity to Relative Changes:
The square root transformation can emphasize smaller differences in intensity, which might be important for detecting subtle changes between spectra.

## improved_similarity_distance
$Distance = \sqrt{\frac{1}{N}\sum\{\frac{P_i-Q_i}{P_i+Q_i}\}^2}$

$N$ is the number of data points (peaks) being compared. This formula is similar to a normalized Euclidean distance but includes a normalization factor.

### Application notes
#### Emphasis on Proportional Differences:
The formula gives more weight to the relative differences between the peaks rather than their absolute differences, which can be useful in contexts where the scale of intensity values is less important than their relative changes.

#### Robustness:
This measure can be more robust to variations in the magnitude of spectral data, as it focuses on the ratio of differences to the sum of intensities.




## inner_product_distance
$ Distance = 1-\sum{P_iQ_i}$

By summing the products of corresponding values, the formula emphasizes the importance of matching peaks with similar intensities.

## intersection_distance
$Distance = 1-\frac{\sum\min{(P_{i},Q_{i})}}{\min(\sum{P_{i},\sum{Q_{i})}}}$

is a variation of the [tanimoto coefficient](https://en.wikipedia.org/wiki/Jaccard_index).

This formula calculates a similarity measure by summing the minimum of corresponding intensity values from two spectra and then normalizing this sum by the minimum of the total sums of intensities from both spectra.

### Application notes
#### Focus on Common Elements:
By using the minimum of corresponding values, the formula emphasizes the presence of common peaks with similar intensities in both spectra.

#### Robustness:
This measure can be more robust to differences in the scale of the spectra, as it focuses on the overlap of intensities rather than their absolute values.

## jaccard_distance
$Distance = \frac{\sum(P_i-Q_i)^2}{\sum P_i^2+\sum{Q_i^2-\sum{P_iQ_i}}}$

is similar to the cosine metric, but it includes an additional term that subtracts the product of the corresponding intensities.
This formula calculates a distance measure by summing the squared differences between corresponding intensity values from two spectra, then dividing by the sum of the squares of the intensities minus the sum of the products of corresponding intensities. This could be interpreted as a measure that takes into account not only the differences in intensities but also the interaction between the intensities.
### Application notes
#### Interaction Effect:
By including the term $\sum{P_iQ_i}$,the formula accounts for the interaction between the spectra, which could provide a more measure of similarity.

## lorentzian_distance
$Distance = \sum{\ln(1+|P_i-Q_i|)}$

calculates a sum that includes the natural logarithm of one plus the absolute difference between corresponding intensity values from two spectra. The natural logarithm is used to reduce the impact of large differences and provide a more gradual change as differences increase.

### Application notes
#### Emphasis on Large Differences:
The logarithmic transformation gives more weight to large differences between corresponding spectral points, which can be useful in emphasizing significant discrepancies.

## manhattan_distance
$Distance = \sum|P_{i}-Q_{i}|$

calculates the sum of the absolute differences between each pair of corresponding values from two spectra. The result gives a raw measure of how far apart the two spectra are, in terms of the total deviation across all corresponding points.

### Application notes
#### Robustness:
The use of absolute values makes this measure robust to outliers, as it does not amplify the effect of large deviations as squaring does.

####Lack of Weighting:
Unlike some other metrics, this formula does not weight differences by their importance or by the scale of the data, treating all differences equally.

## matusita_distance
$Distance = \sqrt{\sum(\sqrt{P_{i}}-\sqrt{Q_{i}})^2}$

calculates the Euclidean distance between the square root transformed values of two spectra. It transforms each intensity value by taking the square root and then computes the standard Euclidean distance between these transformed vectors.

### Application notes
#### Sensitivity to Proportional Differences:
By taking the square root of the intensities, the formula gives more weight to smaller differences and less to larger ones, which can be beneficial when the scale of intensity values is important.

## mean_character_distance
$Distance = \frac{1}{N}\sum{|P_i-Q_i|}$

 sums the absolute differences between corresponding points and divides by the number of data points $N$, providing a measure of central tendency that indicates the average dissimilarity between the two spectra.

 ### Robustness:
 Mean absolute difference is less sensitive to outliers compared to measures that involve squaring differences, such as the Euclidean distance.

 ### Sensitivity to Distribution:
 The formula normalizes the sum of differences, allowing for comparison between spectra of different sizes or with different numbers of data points.
 However, the mean difference does not take into account the distribution of the data, which could be important in certain applications.

## motyka_distance
$Distance = -\frac{\sum\min{(P_{i},Q_{i})}}{\sum(P_{i}+Q_{i})}$

$\sum\min{(P_{i},Q_{i})}$ calculates the sum of the minimum values between corresponding elements of the two spectra. This captures the overlap or commonality between the two sets of data points.

${\sum(P_{i}+Q_{i})}$ calculates the sum of all values in both spectra combined, representing the total presence or coverage of data points across both spectra.

The ratio represents the proportion of overlap between the two spectra.

### Application notes
#### Balanced Measure:
This formula balances the contribution of common and unique elements in both spectra, providing a comprehensive measure of similarity.

#### Normalization:
By considering the sum of all elements in both spectra, it implicitly normalizes the measure, making it less sensitive to differences in the scale of data.

## pearson_correlation_distance
$Distance = \frac{\sum[(Q_i-\bar{Q})(P_i-\bar{P})]}{\sqrt{\sum(Q_i-\bar{Q})^2\sum(P_i-\bar{P})^2}}$

${\sum[(Q_i-\bar{Q})(P_i-\bar{P})]}$ calculates the sum of the products of the deviations of each corresponding value from its mean. This captures the linear relationship between the two sets of data.

${\sqrt{\sum(Q_i-\bar{Q})^2\sum(P_i-\bar{P})^2}}$ normalizes the numerator by the product of the standard deviations of each set, ensuring that the result is independent of the scale of the data.

### Application notes
#### Sensitivity to Linear Relationships:
This measure is sensitive to linear relationships between the two sets of data, making it useful for identifying correlated changes in spectral data.

#### Normalization:
By normalizing the covariance, the formula provides a measure that is not affected by the magnitude of the data, focusing instead on the pattern of relationship.



## penrose_shape_distance
$Distance = \sqrt{\sum((P_i-\bar{P})-(Q_i-\bar{Q}))^2}$

$(P_i-\bar{P})$ and $(Q_i-\bar{Q})$ represent the deviations of each element from its respective mean. Centering the data in this way helps to focus on the relative differences rather than the absolute values.

Squaring these differences emphasizes larger disparities and minimizes the effect of smaller ones. Summing these squared differences gives a total measure of similarity.

### Application notes
#### Focus on Relative Differences:
By centering the data, the formula focuses on relative differences between the vectors, which can be more meaningful than absolute differences when the scales of the data vary.



## penrose_size_distance
$Distance = \sqrt N\sum{|P_i-Q_i|}$

calculates the sum of the absolute differences between the intensities of corresponding ions in the two spectra, scaled by the square root ($\sqrt N$) of the total number of ions.

### Applications
#### Centering:
The use of absolute differences helps to focus on the relative differences rather than the absolute values.

#### Robustness:
The use of absolute differences makes the algorithm robust to noise and variations in ion intensities. It does not assume a specific distribution of the data, which can be advantageous when dealing with real-world mass spectrometry data that may not follow a Gaussian distribution

## probabilistic_symmetric_chi_squared_distance

$Distance = \frac{1}{2} \times \sum\frac{(P_{i}-Q_{i}\ )^2}{P_{i}+Q_{i}\ }$

The squared differences $(P_{i}-Q_{i})^2$ are weighted by $(P_i + Q_i)$, which helps to normalize the differences and reduce the impact of very extreme intensity. Summing these weighted squared differences gives a total measure of dissimilarity.

### Application notes
#### Robustness:
The use of squared differences makes the algorithm sensitive to larger disparities, but the normalization by the sum of intensities helps to balance this sensitivity.

## roberts_distance
$Distance = 1-\sum\frac{(P_{i}+Q_{i})\frac{\min{(P_{i},Q_{i})}}{\max{(P_{i},Q_{i})}}}{\sum(P_{i}+Q_{i})}$

$\frac{\min{(P_{i},Q_{i})}}{\max{(P_{i},Q_{i})}}$ represents the ratio of the minimum intensity to the maximum intensity for each ion pair.

The weighted sum $(P_{i}+Q_{i})$ gives more weight to ions with higher intensities, making the similarity measure more sensitive to significant peaks.

The denominator $\sum(P_{i}+Q_{i})$normalizes the weighted sum, ensuring that the similarity score is comparable across different spectra.

### Application notes
#### Focus on Relative Intensities:
The algorithm focuses on the relative intensities of corresponding ions, which can be more meaningful than absolute differences when comparing mass spectra.

#### Weighted Contribution:
The weighted sum ensures that ions with higher intensities contribute more to the similarity score, reflecting their significance in the spectra.

#### Normalization:
The normalization by the sum of intensities makes the similarity score comparable across different spectra, regardless of their overall intensity levels.




## ruzicka_distance
$Distance = \frac{\sum{|P_{i}-Q_{i}|}}{\sum{\max(P_{i},Q_{i})}}$

Summing these absolute differences ($|P_i-Q_i|$) gives a total measure of dissimilarity.

The denominator ${\sum{\max(P_{i},Q_{i})}}$ normalizes the sum of absolute differences, ensuring that the similarity score is comparable across different spectra.

### Application notes
#### Focus on Relative Differences:
The algorithm focuses on the relative differences between the intensities of corresponding ions, which can be more meaningful than absolute differences when comparing mass spectra.

#### Normalization:
The normalization by the sum of maximum intensities makes the similarity score comparable across different spectra, regardless of their overall intensity levels.

#### Robustness:
It does not assume a specific distribution of the data, which can be advantageous when dealing with real-world mass spectrometry data that may not follow a Gaussian distribution.

## spectral_contrast_angle_distance
$Distance = 1 - \frac{\sum{Q_iP_i}}{\sqrt{\sum Q_i^2\sum P_i^2}}$

${\sum{Q_iP_i}}$ represents the dot product of the intensity from two spectra and ${\sqrt{\sum Q_i^2\sum P_i^2}}$ normalizes the dot product, ensuring that the similarity score is comparable across different spectra.

### Application notes
#### Normalization:
The algorithm normalizes the dot product by the magnitudes of the spectra, making the similarity score comparable across different spectra.
#### Robustness:
The cosine similarity is robust to variations in the magnitudes of the spectra, focusing on the direction of the vectors rather than their magnitudes.

## squared_chord_distance
$Distance = \sum(\sqrt{P_{i}}-\sqrt{Q_{i}})^2$

$\sqrt{P_{i}}$ and $\sqrt{Q_{i}}$ the square roots of the intensities of the ions. This transformation helps to reduce the impact of very intense ions and makes the measure more comparable across different data sets.

Squaring the differences ($(\sqrt{P_{i}}-\sqrt{Q_{i}})^2$) emphasizes larger disparities and minimizes the effect of smaller ones. Summing these squared differences gives a total measure of dissimilarity.

### Application notes
#### Robustness:
The square root transformation reduces the impact of extreme intensity, making the similarity measure more comparable across different spectra. The use of squared differences makes the algorithm sensitive to larger disparities, but the square root transformation helps to balance this sensitivity.

#### Sensitivity to Outliers:
Like standard Euclidean distance, this measure can be sensitive to outliers because squaring differences amplifies the impact of large deviations.


## squared_euclidean_distance
$Distance = \sum(P_{i}-Q_{i})^2$

calculates the sum of the squared differences between the intensities of corresponding ions.

### Application notes
#### Robustness:
The use of squared differences makes the algorithm sensitive to larger disparities, which can be useful in identifying significant differences between spectra.

#### Sensitivity to Outliers:
The algorithm is sensitive to outliers because squaring differences amplifies the impact of large deviations.

## symmetric_chi_squared_distance
$Distance =  \sqrt{\sum{\frac{\bar{P}+\bar{Q}}{N(\bar{P}+\bar{Q})^2}\frac{(P_i\bar{Q}-Q_i\bar{P})^2}{P_i+Q_i}\ }}$

$\frac{(P_i\bar{Q}-Q_i\bar{P})^2}{P_i+Q_i}$ represents the squared difference between the intensities of corresponding ions, weighted by the sum of the intensities of the ions.

${\sum{\frac{\bar{P}+\bar{Q}}{N(\bar{P}+\bar{Q})^2}}}$ normalizes the weighted squared differences by the mean intensities of the spectra and the total number of ions, ensuring that the similarity score is comparable across different spectra.

Taking the square root of the sum provides a distance measure, which is more intuitive and comparable across different data sets.

### Application notes
#### Normalization:
The algorithm normalizes the differences by the mean intensities and the total number of ions, making the similarity score comparable across different spectra
#### Robustness:
The use of squared differences makes the algorithm sensitive to larger disparities, but the normalization helps to balance this sensitivity

#### Sensitivity to Outliers:
The algorithm is sensitive to outliers because squaring differences amplifies the impact of large deviations



## unweighted_entropy_distance
$Distance = -\frac{2\times S_{PQ}-S_P-S_Q} {ln(4)}, S_I=\sum_{i} {I_i ln(I_i)}$

$S_I=\sum_{i} {I_i ln(I_i)}$ represents the Shannon entropy of the spectrum $I$, which measures the uncertainty or information content of the spectrum.

$S_{PQ}$ represents the joint entropy of the two spectra, which measures the combined uncertainty of the two spectra.
The formula calculates the calculates the similarity between the two spectra based on their joint and individual entropies. The normalization by ${ln(4)}$ ensures that the similarity score is comparable across different spectra.

### Application notes
#### Information Theoretic Measure:
The algorithm uses Shannon entropy, which is an information theoretic measure that captures the information content of the spectra. This can provide a more meaningful measure of similarity compared to simple intensity differences.

#### Robustness:
The use of entropy makes the algorithm robust to variations in the magnitudes of the spectra, focusing on the distribution of intensities rather than their absolute values.


## vicis_symmetric_chi_squared_3_distance
$Distance = \sum\frac{(P_i-Q_i)^2}{\max{(P_i,Q_i)}}$

calculates the sum of the squared differences between the intensities of corresponding ions, normalized by the maximum intensity for each pair.

### Application notes
#### Limited Statistical Significance:
The algorithm does not provide a statistical significance measure for the similarity score, which can make it difficult to determine whether the observed similarity is due to chance or a true match.

#### Normalization & Robustness

## wave_hedges_distance
$Distance = \sum\frac{|P_i-Q_i|}{\max{(P_i,Q_i)}}$

calculates the sum of the absolute differences between the intensities of corresponding ions, normalized by the maximum intensity for each ion pair.
### Application notes
#### Normalization & Robustness
#### Limited Statistical Significance

## whittaker_index_of_association_distance
$Distance = \frac{1}{2}\sum|\frac{P_i}{\bar{P}}-\frac{Q_i}{\bar{Q}}|$

$\frac{P_i}{\bar{P}}$ and $\frac{Q_i}{\bar{Q}}|$ represent the normalized intensities of the ions, where each intensity is divided by the mean intensity of its respective spectrum.

This algorithm calculates the sum of the absolute differences between the normalized intensities of corresponding ions, scaled by a factor of $\frac{1}{2}$

### Application notes
#### Normalization & Robustness
#### Limited Statistical Significance

## ms_for_id_distance
$Distance = -\frac{N_m^b(\sum I_{q,i}+2\sum I_{r,i})^c}{(N_q+2N_r)^d+\sum|I_{q,i}-I_{r,i}|+\sum|M_{q,i}-M_{r,i}|},\ \ b=4,\ c=1.25,\ d=2
$

The peaks have been filtered with intensity > 0.05.

$N_m$: number of matching fragments,

$N_q, N_r$: number of fragments for spectrum p,q,

$M_q,M_r$: m/z of peak in query and reference spectrum,

$I_q,I_r$: intensity of peak in query and reference spectrum

$N_m^b$ is a normalization factor that scales the similarity score.

$\sum I_{q,i}+2\sum I_{r,i}$ represents the sum of the intensities of the ions in the two spectra, with the second spectrum's intensities weighted by a factor of 2.

$\sum|M_{q,i}-M_{r,i}|$ represents the sum of the differences in *m/z* ratios of corresponding ions.

$\sum|I_{q,i}-I_{r,i}|$ represents the sum of the absolute differences in intensities of corresponding ions.

The denominator normalizes the similarity score by the total number of ions and the differences in intensities and mass-to-charge ratios. The exponent $d=2$ amplifies the effect of the total number of ions.

### Application notes
#### Normalization:
The algorithm normalizes the similarity score by the total number of ions and the differences in intensities and mass-to-charge ratios, making the score comparable across different spectra

#### Robustness:
The use of absolute differences and the normalization by the total number of ions make the algorithm robust to variations in the magnitudes of the spectra







## ms_for_id_v1_distance
$Similarity = \frac{N_m^4}{N_qN_r(\sum|I_{q,i}-I_{r,i}|)^a}\ ,\ a=0.25
$

$Distance = \frac{1}{Similarity}$

$N_m$: number of matching fragments

$N_q, N_r$: number of fragments for spectrum p,q

$\frac{N_m^4}{N_qN_r}$ normalizes the similarity score by the number of matching fragments and the total number of fragments in the two spectra. The exponent 4 on $N_m$ amplifies the effect of the number of matching fragments.

$\sum|I_{q,i}-I_{r,i}|$ represents the sum of the absolute differences in intensities of corresponding ions.

The exponent $\ a=0.25$ reduces the impact of the intensity differences, making the similarity score less sensitive to large differences in intensities.

### Application notes
#### Normalization:
The algorithm normalizes the similarity score by the number of matching fragments and the total number of fragments, making the score comparable across different spectra.

#### Robustness:
The use of absolute differences and the exponent $\ a$ makes the algorithm robust to variations in the magnitudes of the spectra.


## weighted_dot_product_distance
$Distance = 1 - \frac{(\sum{Q^{'}_{i} P^{'}_{i}})^2}{\sum{Q_{i}^{'2}\sum P_{i}^{'2}}}$
here: $P^{'}_{i} = M_{p,i}^{3}I_{p,i}^{0.6}, Q^{'}_{i} = M_{q,i}^{3}I_{q,i}^{0.6}$

$P^{'}_{i}$ and $Q^{'}_{i}$ represent the transformed intensities of the ions, where the mass-to-charge ratios are raised to the power of 3 and the intensities are raised to the power of 0.6. This transformation helps to emphasize the contribution of ions with higher mass-to-charge ratios and moderate intensities.

And the subsequent calculation is the same as dot_product_distance.




## peak_percentage

$Simlarity = \frac{N_m}{N_p}$

$N_m$: number of matching fragments,

$N_p, N_r$: number of fragments for spectrum p

is calculated as the ratio of the number of common fragments to the total number of fragments in the query spectrum.

### Application notes
#### Sensitivity to Fragmentation:
The algorithm is straightforward to just consider the fragmentation pattern of the spectra and overlook the influence of intensities.

In tandem mass spectra preprocessing, signals ≤ 1% intensity of the highest peak are usually considered noise. After a manual inspection of MNA’s experimental mass spectral library, some essential fragment ions still exist, mixed in with amounts of noise signals even after employing this noise-removing operation. It is an obstacle to deciding a general standard for noise-removing. There is another part of MS2 spectra predicted by CFM-ID in MNA’s library. Although the CFM-ID model is well-designed and applies state-of-the-art machine learning modules, it is impossible for in-silico fragments to follow real-world cleavage laws. Unexpected simulated fragments will undoubtedly affect the comparison of $ MS^2 $ spectra.
Considering the reasons mentioned above, we designed a new mass spectral similarity algorithm called peak percentage (Supplementary Fig. 3). It counts all matched peaks without a user-defined mass error, and the count is divided into the total number of peaks to get the final score. In practical use, we advise using this scoring method after the $MS1$ filtering procedure. The absolute number of matched peak counts (MPC) was also used synergistically to restrict meaningless comparisons of oversimplified spectra.


## Neutral loss
Neutral loss is the difference in *Δm/z* between a precursor ion and its fragment ions in MS/MS spectra, effectively providing insight into the neutral molecules lost during ion fragmentation.

### Application notes
#### Additional dimension:
Neutral loss adds a valuable dimension to MS<sup>2</sup> analysis, particularly for structurally related compounds sharing the common building blocks.

## Modified cosine
Modified cosine similarity builds on traditional cosine similarity. It matches fragment ions directly and also accounts for peaks shifted by the precursor mass difference between spectra, capturing structural changes due to molecular modifications.

### Application notes
It incorporates precursor mass differences, enabling it to detect fragment ions altered by structural modifications.  While effective for single modifications.


## Reference
See [J. Am. Soc. Mass Spectrom. 2022, 33, 9, 1733–1744](https://pubs.acs.org/doi/10.1021/jasms.2c00153) and [J. Am. Soc. Mass Spectrom. 2022, 33, 3, 530–534](https://pubs.acs.org/doi/10.1021/jasms.1c00343) for details


